# Single-Order Spectrum

This will show how to fit a single-order spectrum using our [previous setup](setup.ipynb) on [WASP14](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=wasp+14&submit=SIMBAD+search) TRES data.

In [1]:
# TODO